# Pseudo table

In [13]:
import pandas as pd
import numpy as np

## Razvanette

In [14]:
data_model = pd.read_csv('SEER_clean_datav1210.csv', low_memory=False)

In [15]:
#Death All Cause
data_model.loc[data_model['Vital_status_rec'] == 'Dead' ,'Death_All_Cause'] = 1
data_model.loc[data_model['Vital_status_rec'] != 'Dead' , 'Death_All_Cause'] = 0
data_model = data_model.drop(columns='Vital_status_rec')
data_model['Death_All_Cause'].value_counts(dropna=False), print('Mortality All causes :',  data_model['Death_All_Cause'].sum() /  data_model.shape[0])

Mortality All causes : 0.2068389505748256


(0.0    80720
 1.0    21050
 Name: Death_All_Cause, dtype: int64,
 None)

In [16]:
# Event if death of any cause 0==Alive
data_model['IS_DEAD'] =  data_model['Cod_site'] != 'Alive'
print('Dead number:',  data_model['IS_DEAD'].sum())
print('Dead number all cause:',  data_model['Death_All_Cause'].sum())
print('Global Mortality :',  data_model['IS_DEAD'].sum() /  data_model.shape[0])

Dead number: 21050
Dead number all cause: 21050.0
Global Mortality : 0.2068389505748256


In [17]:
#Exposure derivation
def exposure_expansion(dataset):
    
    #Yearly survival time (optional)
    dataset.loc[:, 'SRV_YR'] = (dataset['Surv_mth']+1)/12
    
    #Exposure calculation

    #Number of durations
    dataset.loc[:, 'DURATION_MAX'] = np.ceil(dataset['SRV_YR']).astype('int')
    #Expansion of durations per columns
    dataset.loc[:, 'DURATIONS'] = dataset['DURATION_MAX'].apply(range)
    dataset_expanded = dataset['DURATIONS'].apply(pd.Series)
    #Stack all the durations (by row)
    dataset_expanded = dataset_expanded.stack()
    #Index cleaning
    dataset_expanded.index = dataset_expanded.index.droplevel(1)
    dataset_expanded.name = 'DURATION'
    #Add features
    dataset_expanded = pd.concat([dataset_expanded, dataset], axis = 1)
    #Exposure calculation
    dataset_expanded.loc[:, 'EXPO'] = dataset_expanded['SRV_YR'] - dataset_expanded['DURATION']
    dataset_expanded.loc[dataset_expanded['EXPO'] > 1, 'EXPO'] = 1
    dataset_expanded.loc[:, 'DURATION_LAST'] = dataset_expanded['DURATION_MAX']-1 == dataset_expanded['DURATION']
    
    #Set exposure of last duration to 1 if the paient is dead
    dataset_expanded.loc[dataset_expanded['DURATION_LAST'] & dataset_expanded['IS_DEAD'], 'EXPO'] = 1
    
    # Computation of duration when death occured
    dataset_expanded.loc[:, 'IS_DEAD_DURATION'] = dataset_expanded['DURATION_LAST'] & dataset_expanded['IS_DEAD']

    # Year
    # Assumption : duration begins on the 01/01/xxxx
    dataset_expanded.loc[:, 'YEAR'] = dataset_expanded['Yr_dx'] + dataset_expanded['DURATION']

    # Age
    # Assumption : patient is born at beginning of duration
    dataset_expanded.loc[:, 'AGE'] = dataset_expanded['Age_dx'] + dataset_expanded['DURATION']

    return dataset_expanded

In [18]:
df_expansion = exposure_expansion(data_model)

In [19]:
## Cause Of Death is Melanoma of the Skin ? True or False : 
df_expansion['Death_skin'] = 0
df_expansion.loc[(df_expansion['Cod_site'] == 'Melanoma of the Skin') & (df_expansion['DURATION_LAST'] == 1) ,'Death_skin'] = 1
df_expansion['Death_skin'].value_counts(dropna=False)

0    622210
1      7343
Name: Death_skin, dtype: int64

In [20]:
## Cause Of Death is other ? True or False :  
df_expansion.loc[(df_expansion['Death_All_Cause'] == 1 ) & (df_expansion['DURATION_LAST'] == 1),'Death_All_Cause_ext' ] = 1
df_expansion.loc[(df_expansion['Death_All_Cause'] != 1 ) | (df_expansion['DURATION_LAST'] == 0),'Death_All_Cause_ext'] = 0
df_expansion = df_expansion.drop(columns='Death_All_Cause')
df_expansion['Death_All_Cause_ext'].value_counts(dropna=False)

0.0    608503
1.0     21050
Name: Death_All_Cause_ext, dtype: int64

In [21]:
# don't need to keep all variables
df_expansion = df_expansion.drop(columns=['patient_ID', 'rec_nb', 'IS_DEAD', 'IS_DEAD_DURATION', 'DURATION_MAX', 'DURATIONS', 'SRV_YR', 'Surv_mth', 'MIN_REC_NO', 'DURATION_LAST', 'Cod_site'])

In [22]:
min(df_expansion['AGE']), max(df_expansion['AGE'])

(0.0, 107.0)

In [23]:
# keep ages between 18 and 80
df_expansion = df_expansion[(df_expansion['AGE'] >= 18) & (df_expansion['AGE'] <= 80)]
df_expansion.shape

(545250, 28)

In [24]:
# only have info until 2018 --> delete YEAR > 2018
df_expansion = df_expansion[df_expansion['YEAR'] <= 2018]

In [ ]:
df_expansion.shape

In [ ]:
df_expansion.columns

In [25]:
df_expansion.to_csv('SEER_expanded_v1210.csv', index=False)